In [87]:
import codecs
from collections import defaultdict

definitions = defaultdict(list)
with codecs.open(r'locale\ru\vocab\ozhegov.txt', 'r', 'utf8') as f:
    description = f.readline().split('|')
    states = [line.split('|') for line in f]
    for s in states:
        definitions[s[0]].append(s)

In [6]:
print (description)

['VOCAB', 'BASEFORM', 'PHONGL', 'GRCLASSGL', 'STYLGL', 'DEF', 'ANTI', 'LEGLEXAM\r\n']


In [88]:
for states in definitions.values():
    for i in range(1,len(states)):
        if states[i-1][4].startswith('!'):
            atakze = states[i][4].rfind(u'а также')
            s4 = states[i][4][atakze:] if atakze > 0 else states[i][4]
            states[i][4] =states[i-1][4][1:]+' '+ states[i-1][5]+', '+ s4

In [89]:
import re

abbrs = {r'\bobs\.?': u'устаревшее',
        r'\bустар\.': u'устаревшее',
        r'\bстар\.': u'в старину',
        r'\bColloq\.?': u'разговорное',
        r'\bразг\.': u'разговорное',
        r'\bшутл\.': u'шутлилое',
        r'\bофиц\.': u'официальное',
        r'\bвысок\.': u'высокое',
        r'\bупотр\.': u'употребляется',
        r'\bобл\.': u'местное',
        r'\bпрост\.': u'просторечное',
        r'-н\.': u'-нибудь',
        r'-л\.': u'-либо',
        r'\bнеодобр\.': u'неодобрительное',
        r'\bnon-st\.': u'',
        r'\bсо знач\.': u'со значением',
        r'\bперен\.': u'переносное',
        r'\bв знач\.': u'в значении',
        r'\bSpec': u'',
        r'\bLib': u'',
        r'\bирон\.': u'ироничное',
        r'\bспец\.': u'специальное',
        r'\bкнижн\.': u'книжное',
       }

for states in definitions.values():
    for s in states:
        for (abbr,value) in abbrs.items():
            for i in [4,5]:
                s[i] = re.sub(abbr, value, s[i], flags=re.IGNORECASE)
                s[i] = re.sub(r'\(-\w+\)', '', s[i])  #изобразить (-ать)
                s[i] = re.sub(r'==', u'то же, что и ', s[i])
                s[i] = re.sub(r'<=', 'от слова ', s[i])
                s[i] = re.sub(r'\d*\s*N\d*\W*\d*','', s[i], flags=re.IGNORECASE)
                s[i] = re.sub(r'[^а-яё0-9\",\.:\(\)\s-]+', '', s[i], flags=re.IGNORECASE)
                s[i] = re.sub(r'\(\s+', '(', s[i])
                s[i] = re.sub(r'\s+\)', ')', s[i])
                s[i] = re.sub(r'\s+', ' ', s[i])

In [92]:
with codecs.open(r'locale\ru\vocab\ozhegov.hint.txt', 'w+', 'utf8') as f:
    for word in sorted(definitions.keys()):
        for s in definitions[word]:
            worddef = (s[4]+' '+s[5]).strip()
            if worddef != "":
                f.write(s[0]+'|'+worddef+'\n')
        

Parsing orphography dictionary of Lopatin (to get nouns)

In [62]:
import pymorphy2
import codecs

morph = pymorphy2.MorphAnalyzer()

nouns = set()
adjvs = set()
adjends = [u'ый', u'ний', u'кий', u'ой', u'ая', u'яя', u'ое', u'ее']

def add(line):
    if not any([line.endswith(end) for end in adjends]):
        nouns.add(line)
    else:
        adjvs.add(line)

with codecs.open(r'locale\ru\vocab\dicts\lopatin.txt', 'r', 'utf8') as f:
    for line in f:
        line = line.strip()
        if len(line) == 0 or line[0].isupper() or ' ' in line:
            continue
        line = line.lower()
        if (line.endswith(u'цы') and line[0:-2]+u'ец' in nouns):
            continue
        forms = morph.parse(line)
        for form in forms:
            if form.score > 0.01:
                if ({'NOUN', 'nomn', 'sing'} in form.tag or 
                    ({'NOUN', 'nomn', 'plur'} in form.tag and {'NOUN', 'nomn', 'plur'} in form.normalized.tag)):
                    add(line)
                    break

In [63]:
with codecs.open(r'locale\ru\vocab\dicts\lopatin.noun.txt', 'w+', 'utf8') as f:
    for noun in sorted(set([n.replace('ё', 'е') for n in nouns])):
        f.write(noun + '\r\n')
with codecs.open(r'locale\ru\vocab\dicts\lopatin.noun-adj.txt', 'w+', 'utf8') as f:
    for word in sorted(set([n.replace('ё', 'е') for n in adjvs])):
        f.write(word + '\r\n')

In [24]:
import os
import codecs
zaliznyakdir = r'C:\StarSoft\dict\Z'
filenames = next(os.walk(zaliznyakdir))[2]
with codecs.open(r'locale\ru\vocab\zaliznyak.txt', 'w+', 'utf8') as fout:
    for filename in filenames:
        with codecs.open(os.path.join(zaliznyakdir, filename), 'r', 'cp866') as f:
            fout.write(f.read()+'\n')


In [25]:
import re
wregex = re.compile(r'^([\w-]+)\s\d\s([^\d]+)\d')
lines = []
with codecs.open(r'locale\ru\vocab\zaliznyak.txt', 'r', 'utf8') as f:
    for line in f:
        m = wregex.match(line)
        if m:
            lines.append([m.group(1), m.group(2).strip()])

In [39]:
t = set([line[1] for line in lines])
#print (t)
nountype = {'м//с', 'м//ж', 'мо жо', 'мо-жо', 'м//мо', 'жо', 'с//мн. неод.', 'ж', 'с',
            'жо//ж','мо', 'м//ж, ж', 'ж//жо', 'мо-жо жо', 'мн. мо', 'с//ж', 'мн. м', 
            'со', 'мн. ж', 'мн. с', 'м', 'мо//жо, жо', 'с//м', 'мо//со, со', 'мо со', 'мн. жо', 'мо//жо'}
mbnountype = {'жо п',  'м с', 'мн. со'}
mn = {'мн. _от_ м', 'мн. _от_ с', 'мн. _от_ ж'}

In [40]:
with codecs.open(r'locale\ru\vocab\zaliznyak.noun.txt', 'w+', 'utf8') as f:
    for line in lines:
        if line[1] in nountype:
            f.write(line[0]+'\n')

with codecs.open(r'locale\ru\vocab\zaliznyak.mb_noun.txt', 'w+', 'utf8') as f:
    for line in lines:
        if line[1] in mbnountype or (line[1] not in mn and line[1].startswith('мн. _от_')):
            f.write(line[0]+'\n')

In [ ]:
with codecs.open(r'locale\ru\vocab\efremova.txt')

In [55]:
word_groups = [[]]
with codecs.open(r'locale\ru\vocab\dicts\hagen_fullparadigm.txt') as f:
    for line in f:        
        
        if line.strip() == '':
            word_groups.append([])
        if line.startswith(' '):
            continue
        word_groups[-1].append(line.split('|'))

In [74]:
from collections import defaultdict

posofword = defaultdict(set)
for wgroup in word_groups:
    keyword = wgroup[0][0].strip()
    if ' ' in keyword:
        continue
    posofword[keyword].add(wgroup[0][1].split()[0])

nouns = []    
for wgroup in word_groups:
    keyword = wgroup[0][0].strip()
    if ' ' in keyword:
        continue
    if  ' сущ ' in wgroup[0][1] and 'прл' not in posofword[keyword]:
        is_sign = ' ед ' in wgroup[0][1]
        for word in wgroup:
            if not is_sign or ' ед ' in word[1]:
                nouns.append(word[0].strip())
        

In [75]:
with codecs.open(r'locale\ru\vocab\dicts\hagen_fullparadigm.noun.txt', 'w+', 'utf8') as f:
    for noun in nouns:
        f.write(noun+'\n')
    

In [71]:
sets = []
with codecs.open(r'locale\ru\vocab\dicts\hagen_fullparadigm.noun.txt', 'r', 'utf8') as f:
    sets.append(('hagen', set([word.strip() for word in f])))
with codecs.open(r'locale\ru\vocab\dicts\lopatin.noun.txt', 'r', 'utf8') as f:
    sets.append(('lopatin', set([word.strip() for word in f])))
with codecs.open(r'locale\ru\vocab\dicts\zaliznyak.noun.txt', 'r', 'utf8') as f:
    sets.append(('zaliznyak', set([word.strip() for word in f])))

for s1 in sets:
    for s2 in sets:
        if s1[0] == s2[0]:
            continue
        print('In '+s1[0]+' but not in '+s2[0]+':')
        diff = list(s1[1] - s2[1])
        print(' '.join(diff[0:30]))
        print(len(diff))
        print('')



In hagen but not in lopatin:
орала проститут молодчество черниленка ремнецветные заверточка курсовая бактероид отгребщик ларго неметчина тестик волюминометр примащивание отсырелость полосочка хирагрик залеживание метатония разгрузчица нанимание флицид трематодоз сдельные элизий похудание малорослость ногай наживщик завербованный
15234

In hagen but not in zaliznyak:
проститут молодчество масленок бактероид отгребщик религиоведение барнаулка сутемь залеживание парогенератор дискетка ногай неокапитализм пирующая территориальность иберка унтер-офицерство перелицовывание родословная выколотчик лизосома лейтенантик долихоцефалия многоногие подмазывание эксгибиционизм переделывательница вручительница вьюновые исчезание
27375

In lopatin but not in hagen:
молибденирование телеангиэктазия сортоучасток технократичность чикагец чур-чура балашихинец аэрогеология фильмохранилище ковеллин киногруппа бессилеть устраивание гидрорежим высокосортность моноспорангий приимец скаутинг-агентство лагман бух